In [21]:
# Import packages
import os
import random
from PIL import Image
from torch import nn
import torch 
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import json 


from src.config import PROCESSED_DATA_DIR, RAW_DATA_DIR

image_path = PROCESSED_DATA_DIR / "pizza_hamburger_hotdog_10_percent"
train_dir = image_path / 'train' 
test_dir = image_path / 'test'

device ='cuda' if torch.cuda.is_available() else 'cpu'

In [22]:
config_path = 'params.json'

with open(config_path, 'r') as config_file:
   config = json.load(config_file)
#print(json.dumps(config, indent=2))

In [23]:
# Load hyperparams from params.json

learning_rate = config['model_params']['learning_rate']
batch_size = config['model_params']['batch_size']
num_epochs = config['model_params']['num_epochs']
dropout_rate = config['model_params']['dropout_rate']
optimizer = config['model_params']['optimizer']
loss_function = config['model_params']['loss_function']
metrics = config['model_params']['metrics']
output_shape = config['model_params']['output_shape']

In [24]:
# Prepare data into dataloader
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
])

train_data = ImageFolder(train_dir, transform=transform)
test_data = ImageFolder(test_dir, transform=transform)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [25]:
class HHP01(nn.Module):
    def __init__(self, input_shape: int,
                 hidden_units: int,
                 output_shape: int):
        super(HHP01, self).__init__()
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*53*53, out_features=output_shape)
        )
        
    def forward(self, x):
        # x = self.conv_block1(x)
        # print(x.shape)
        # x = self.conv_block2(x)
        # print(x.shape)
        # x = self.classifier(x)
        # print(x.shape)
        # return x
        return self.classifier(self.conv_block2(self.conv_block1(x)))

In [26]:
model = HHP01(input_shape=3, hidden_units=10, output_shape=output_shape).to(device)
model

HHP01(
  (conv_block1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=28090, out_features=3, bias=True)
  )
)

In [27]:
image_batch, label_batch = next(iter(train_loader))
image_batch.shape, label_batch.shape

(torch.Size([64, 3, 224, 224]), torch.Size([64]))

In [28]:
model(image_batch.to(device))

tensor([[ 0.0044, -0.0045, -0.0174],
        [ 0.0093, -0.0079, -0.0187],
        [ 0.0008, -0.0094, -0.0147],
        [ 0.0100, -0.0156, -0.0219],
        [ 0.0053, -0.0086, -0.0207],
        [ 0.0033, -0.0130, -0.0254],
        [ 0.0036, -0.0065, -0.0115],
        [ 0.0040, -0.0175, -0.0107],
        [ 0.0084, -0.0045, -0.0192],
        [ 0.0152, -0.0070, -0.0161],
        [ 0.0054, -0.0112, -0.0156],
        [ 0.0107, -0.0184, -0.0112],
        [ 0.0075, -0.0050, -0.0095],
        [ 0.0077, -0.0057, -0.0165],
        [ 0.0046, -0.0013, -0.0241],
        [ 0.0106, -0.0084, -0.0168],
        [ 0.0096, -0.0094, -0.0158],
        [ 0.0087, -0.0099, -0.0098],
        [ 0.0062, -0.0047, -0.0113],
        [ 0.0064, -0.0060, -0.0112],
        [ 0.0133, -0.0079, -0.0200],
        [ 0.0061, -0.0078, -0.0104],
        [ 0.0151, -0.0104, -0.0188],
        [ 0.0070, -0.0068, -0.0200],
        [ 0.0082, -0.0039, -0.0123],
        [ 0.0124, -0.0202, -0.0069],
        [ 0.0067, -0.0112, -0.0159],
 

In [29]:
import torchinfo
torchinfo.summary(model, input_size=(1, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
HHP01                                    [1, 3]                    --
├─Sequential: 1-1                        [1, 10, 110, 110]         --
│    └─Conv2d: 2-1                       [1, 10, 222, 222]         280
│    └─ReLU: 2-2                         [1, 10, 222, 222]         --
│    └─Conv2d: 2-3                       [1, 10, 220, 220]         910
│    └─MaxPool2d: 2-4                    [1, 10, 110, 110]         --
├─Sequential: 1-2                        [1, 10, 53, 53]           --
│    └─Conv2d: 2-5                       [1, 10, 108, 108]         910
│    └─ReLU: 2-6                         [1, 10, 108, 108]         --
│    └─Conv2d: 2-7                       [1, 10, 106, 106]         910
│    └─MaxPool2d: 2-8                    [1, 10, 53, 53]           --
├─Sequential: 1-3                        [1, 3]                    --
│    └─Flatten: 2-9                      [1, 28090]                --
│    └─Line

In [35]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.3)

def train_model(num_epochs):
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = 100 * correct / total
        print(f'Epoch {epoch + 1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

# Call the training function
train_model(num_epochs=num_epochs)

Epoch 1, Loss: 0.7727, Accuracy: 68.00%
Epoch 2, Loss: 0.7610, Accuracy: 68.00%
Epoch 3, Loss: 0.7589, Accuracy: 65.33%
Epoch 4, Loss: 0.7443, Accuracy: 67.56%
Epoch 5, Loss: 0.7501, Accuracy: 68.00%
Epoch 6, Loss: 0.7558, Accuracy: 67.11%
Epoch 7, Loss: 0.7229, Accuracy: 69.33%
Epoch 8, Loss: 0.7211, Accuracy: 68.89%
Epoch 9, Loss: 0.7161, Accuracy: 71.11%
Epoch 10, Loss: 0.7291, Accuracy: 69.33%
Epoch 11, Loss: 0.7090, Accuracy: 67.56%
Epoch 12, Loss: 0.7154, Accuracy: 70.67%
Epoch 13, Loss: 0.6866, Accuracy: 72.00%
Epoch 14, Loss: 0.6913, Accuracy: 69.78%
Epoch 15, Loss: 0.6852, Accuracy: 72.89%
Epoch 16, Loss: 0.6475, Accuracy: 69.78%
Epoch 17, Loss: 0.6631, Accuracy: 71.11%
Epoch 18, Loss: 0.6312, Accuracy: 74.22%
Epoch 19, Loss: 0.6466, Accuracy: 72.00%
Epoch 20, Loss: 0.6404, Accuracy: 71.56%
Epoch 21, Loss: 0.6238, Accuracy: 73.33%
Epoch 22, Loss: 0.6406, Accuracy: 72.44%
Epoch 23, Loss: 0.6730, Accuracy: 68.44%
Epoch 24, Loss: 0.6194, Accuracy: 72.89%
Epoch 25, Loss: 0.6164, A

In [36]:
model = HHP01(input_shape=3, hidden_units=20, output_shape=output_shape).to(device)